# Investigate Price Correlations w/ Fundamentals

### For each metric of interest, X:

- create by-year metric data, X per share
- calculate the average ratio for price-to-X
- rate the price correlations using the following ways:
    - calculate the correlation between price and X
    - plot the price against the price-per-average-price-to-X-ratio

In [1]:
import json
import requests
import numpy as np
import yfinance as yf
from datetime import datetime

## Determine the earliest dates of price and financials data to include in analysis

In [2]:
# set years interval for analysis
analysis_interval_years = 20

# get the earlist fiscal year accordingly
start_year = datetime.utcnow().year - analysis_interval_years
start_date_quote_history = datetime(start_year, 1, 1)
start_year_financials_history = start_year - 1

print("Start date of quote history data:", start_date_quote_history)
print("Start year of financials history data:", start_year_financials_history)

Start date of quote history data: 2001-01-01 00:00:00
Start year of financials history data: 2000


## Helper functions to pull and prep stock price data

In [3]:
def get_payload_json_quote_history_from_yfinance_df(df_quote_history, header='Close'):
    """
    This helper function gets a cleaned-up payload json/dict from the dataframe of 
    historical quotes, which is returned by yfinance api (yfinance.Ticker('<symbol>').history)
    """
    
    _payload_json = {}
    _df = df_quote_history[header]
    
    # loop through all timestamps in the data:
    for timestamp in dict(_df):
        if not np.isnan(_df[timestamp]):
            _payload_json[timestamp.to_pydatetime()] = _df[timestamp]
            
    return _payload_json


def get_quote_history(symbol, interval='1mo', header='Close', 
                      start=start_date_quote_history.strftime('%Y-%m-%d'),
                      end=datetime.utcnow().strftime('%Y-%m-%d')):
    """
    This helper function pulls raw historical quote data using the YFinance API, 
    and returns the cleaned up data in a dictionary of "<timestamp>: <price>".
    """
    
    # initialize an yfinance object, and get the historical quote data 
    # from the API
    yf_object = yf.Ticker(symbol)
    df_quote_history = yf_object.history(interval=interval, start=start, end=end)
    
    # return a "timestamp": "price" dictionary from the dataframe of historical quotes
    return get_payload_json_quote_history_from_yfinance_df(df_quote_history, 
                                                           header=header)


# testing

aapl_quote_history = get_quote_history('AAPL')
aapl_quote_history

# testing successful

{datetime.datetime(2001, 1, 1, 0, 0): 0.33156415820121765,
 datetime.datetime(2001, 2, 1, 0, 0): 0.2798171639442444,
 datetime.datetime(2001, 3, 1, 0, 0): 0.33838674426078796,
 datetime.datetime(2001, 4, 1, 0, 0): 0.39082425832748413,
 datetime.datetime(2001, 5, 1, 0, 0): 0.305882066488266,
 datetime.datetime(2001, 6, 1, 0, 0): 0.35647955536842346,
 datetime.datetime(2001, 7, 1, 0, 0): 0.28809675574302673,
 datetime.datetime(2001, 8, 1, 0, 0): 0.28441667556762695,
 datetime.datetime(2001, 9, 1, 0, 0): 0.2378058284521103,
 datetime.datetime(2001, 10, 1, 0, 0): 0.2692371904850006,
 datetime.datetime(2001, 11, 1, 0, 0): 0.326580673456192,
 datetime.datetime(2001, 12, 1, 0, 0): 0.3357798755168915,
 datetime.datetime(2002, 1, 1, 0, 0): 0.37901824712753296,
 datetime.datetime(2002, 2, 1, 0, 0): 0.3327138423919678,
 datetime.datetime(2002, 3, 1, 0, 0): 0.3629192113876343,
 datetime.datetime(2002, 4, 1, 0, 0): 0.37211838364601135,
 datetime.datetime(2002, 5, 1, 0, 0): 0.35724544525146484,
 dat

## Helper functions to pull and prep stock fundamentals data

In [4]:
def get_guru_data(symbol, data_type, api_token='b4bbdecd0f955e18a90fee818670dd94:42afdf0e68bee024983a72f8b6ad071d'):
    """
    This helper function pulls data from the GuruFocus API, for the given symbol
    and data type.
    """
    
    base_url = 'https://api.gurufocus.com/public/user/' + api_token + '/stock/'
    constructed_url = base_url + symbol + '/' + data_type
    
    r = requests.get(constructed_url)
    if r.status_code != 200:
        return "Error: the GuruFocus API service failed."
    else:
        return r.json()
    

# testing

aapl_financials_history = get_guru_data('AAPL', 'financials')
aapl_financials_history

# testing successful

{'financials': {'financial_template_parameters': {'ind_template': 'N',
   'REITs': 'N',
   'IsDirect': 'Indirect Method'},
  'annuals': {'Fiscal Year': ['1991-09',
    '1992-09',
    '1993-09',
    '1994-09',
    '1995-09',
    '1996-09',
    '1997-09',
    '1998-09',
    '1999-09',
    '2000-09',
    '2001-09',
    '2002-09',
    '2003-09',
    '2004-09',
    '2005-09',
    '2006-09',
    '2007-09',
    '2008-09',
    '2009-09',
    '2010-09',
    '2011-09',
    '2012-09',
    '2013-09',
    '2014-09',
    '2015-09',
    '2016-09',
    '2017-09',
    '2018-09',
    '2019-09',
    '2020-09',
    'TTM'],
   'Preliminary': [0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0],
   'per_share_data_array': {'Revenue per Share': ['0.469',
     '0.516',
     '0.594',
     '0.691',
     '0.803',
     '0.71',
     '0.502',
     '0.316',


In [5]:
class FinancialMetric(object):
    """
    This class implements financial metrics as objects, along with 
    related operations.
    """
    
    def __init__(self, name, timestamps, start_year, values, numerify_values=True, drop_timestamps=['TTM']):
        """
        Constructor.
        
        Input:
            - "name": name of the metric.
            - "timestamps": a list (or list-like) of strings in the format of "%Y-%m"
            - "start_year": the earliest year in the input data to be used
            - "values": a list (or list-like) of values (could be strings) for the metric,
                        with each value corresponding to the timestamp value of the same position
                        in the input list of timestamps
            - "numerify_values": default=True; 
                                 if True attempt to convert all input values to float values
            - "drop_values": default=['TTM']; 
                             a list of strings to be removed from the input timestamps
                             list, and the associated value will also be removed from the input values 
                             list.
        """
        
        # if requested, ensure all values are converted to numeric values (float)
        if numerify_values:
            _values = [float(value) for value in values]
        else:
            _values = values

        # zip the two lists to form a dictionary
        _data = dict(zip(timestamps, _values))

        # form a new dictionary with pre-specified contraints on the "year" values
        _processed_data = {}
        for timestamp in _data:
            if timestamp in drop_timestamps:
                continue
            _timestamp_obj = datetime.strptime(timestamp, '%Y-%m')
            if _timestamp_obj.year >= start_year:
                _processed_data[_timestamp_obj] = _data[timestamp]
        
        # save the processed timestamps and values
        self.name = name
        self.timestamps = tuple(_processed_data.keys())
        self.values = tuple(_processed_data.values())
        
    def get_raw_data(self):
        """
        This method returns the saved timestamps & values as a dictionary of "timestamp: value" pairs.
        """
        
        return dict(zip(self.timestamps, self.values))
    
    def set_per_share_data(self, *args, **kwargs):
        """
        This method sets the per share values for the current metric.
        """
        
        if 'x_total' in kwargs and 'x_per_share' in kwargs:
            if len(kwargs['x_total']) != len(self.values) or len(kwargs['x_total']) != len(kwargs['x_per_share']):
                raise ValueError('Invalid length(s) of values.')
            else:
                _total_shares = np.array(kwargs['x_total']) / kwargs['x_per_share']
                self.per_share_values = list(np.array(self.values) / _total_shares)
                
    def get_per_share_data(self):
        """
        This method returns the saved timestamps & per share values as a dictionary of
        "<timestamp>: <per share value>"
        """
        
        return dict(zip(self.timestamps, self.per_share_values))
        
        
# testing
aapl_ebit = FinancialMetric(name = 'ebit', 
                            timestamps = aapl_financials_history['financials']['annuals']['Fiscal Year'],
                            start_year = start_year_financials_history,
                            values = aapl_financials_history['financials']['annuals']['income_statement']['EBIT'])
aapl_ebit.get_raw_data()

{datetime.datetime(2000, 9, 1, 0, 0): 1113.0,
 datetime.datetime(2001, 9, 1, 0, 0): -36.0,
 datetime.datetime(2002, 9, 1, 0, 0): 98.0,
 datetime.datetime(2003, 9, 1, 0, 0): 25.0,
 datetime.datetime(2004, 9, 1, 0, 0): 373.0,
 datetime.datetime(2005, 9, 1, 0, 0): 1643.0,
 datetime.datetime(2006, 9, 1, 0, 0): 2453.0,
 datetime.datetime(2007, 9, 1, 0, 0): 4407.0,
 datetime.datetime(2008, 9, 1, 0, 0): 8327.0,
 datetime.datetime(2009, 9, 1, 0, 0): 11740.0,
 datetime.datetime(2010, 9, 1, 0, 0): 18385.0,
 datetime.datetime(2011, 9, 1, 0, 0): 34205.0,
 datetime.datetime(2012, 9, 1, 0, 0): 55763.0,
 datetime.datetime(2013, 9, 1, 0, 0): 50291.0,
 datetime.datetime(2014, 9, 1, 0, 0): 53867.0,
 datetime.datetime(2015, 9, 1, 0, 0): 73248.0,
 datetime.datetime(2016, 9, 1, 0, 0): 62828.0,
 datetime.datetime(2017, 9, 1, 0, 0): 66412.0,
 datetime.datetime(2018, 9, 1, 0, 0): 76143.0,
 datetime.datetime(2019, 9, 1, 0, 0): 69313.0,
 datetime.datetime(2020, 9, 1, 0, 0): 69964.0}

In [6]:
aapl_revenue = FinancialMetric(name='revenue',
                               timestamps=aapl_financials_history['financials']['annuals']['Fiscal Year'],
                               start_year=start_year_financials_history,
                               values=aapl_financials_history['financials']['annuals']['income_statement']['Revenue'])
aapl_revenue_per_share = FinancialMetric(name='revenue per share',
                                         timestamps=aapl_financials_history['financials']['annuals']['Fiscal Year'],
                                         start_year=start_year_financials_history,
                                         values=aapl_financials_history['financials']['annuals']['per_share_data_array']['Revenue per Share'])

In [7]:
aapl_ebit.set_per_share_data(x_total=aapl_revenue.values, x_per_share=aapl_revenue_per_share.values)
aapl_ebit.get_per_share_data()

{datetime.datetime(2000, 9, 1, 0, 0): 0.05521082299887261,
 datetime.datetime(2001, 9, 1, 0, 0): -0.0018594070482938656,
 datetime.datetime(2002, 9, 1, 0, 0): 0.0048300243817485196,
 datetime.datetime(2003, 9, 1, 0, 0): 0.0012284517480264218,
 datetime.datetime(2004, 9, 1, 0, 0): 0.017210532673028144,
 datetime.datetime(2005, 9, 1, 0, 0): 0.06852221663915009,
 datetime.datetime(2006, 9, 1, 0, 0): 0.09982179653119337,
 datetime.datetime(2007, 9, 1, 0, 0): 0.17697570998453901,
 datetime.datetime(2008, 9, 1, 0, 0): 0.32960625216718675,
 datetime.datetime(2009, 9, 1, 0, 0): 0.4621573243211747,
 datetime.datetime(2010, 9, 1, 0, 0): 0.7100316596397087,
 datetime.datetime(2011, 9, 1, 0, 0): 1.3043837818363218,
 datetime.datetime(2012, 9, 1, 0, 0): 2.106771660234621,
 datetime.datetime(2013, 9, 1, 0, 0): 1.9279540810953133,
 datetime.datetime(2014, 9, 1, 0, 0): 2.1995311031483356,
 datetime.datetime(2015, 9, 1, 0, 0): 3.1610265836595857,
 datetime.datetime(2016, 9, 1, 0, 0): 2.8555930420749496

## Helper function to calculate the average Price-to-X ratio

In [8]:
def calc_avg_price_ratio(price_data, per_share_metric_data):
    """
    This function calculates and returns the average price-to-metric ratio.
    
    Input:
        - "price_data" - dictionary of "<timestamp>: <price>"
        - "per_share_metric_data" - dictionary of "<timestamp>: <per share metric value>", 
                                    with timestamps at the year level
    """
    
    # convert the keys of the per share metric dictionary from timestamps to years
    _years = [timestamp.year for timestamp in per_share_metric_data.keys()]
    _metric_values_by_year = dict(zip(_years, list(per_share_metric_data.values())))
    
    # get new dictionaries for timestamps where both price and metric value are available
    _merged_data = {}
    for key in price_data:
        _metric_value = _metric_values_by_year.get(key.year)
        if _metric_value:
            _merged_data[key] = (price_data[key], _metric_value, 
                                 price_data[key]/_metric_value)

    # put all price-to-metric ratios into a list
    _ratios = [_merged_data[key][-1] for key in _merged_data]
    
    return sum(_ratios) / len(_ratios)
    

# testing
calc_avg_price_ratio(aapl_quote_history, aapl_ebit.get_per_share_data())

18.62543669540075

In [9]:
def get_normal_price(per_share_metric_data, avg_price_to_x_ratio):
    """
    This function calculates and returns the "normal price" with respect to 
    a specific income/cash flow metric X:
    
        "normal price" = "historical price" x "historical average Price-to-X ratio"

    Inputs:
        - "per_share_metric_data": dictionary of "<timestamp>: <per share metric value>"
        - "avg_price_to_x_ratio": a numerical value
        
    Output:
        - "normal price": see above
    """
    
    return {timestamp: max(0, per_share_metric_data[timestamp] * avg_price_to_x_ratio) \
            for timestamp in per_share_metric_data}


# testing
aapl_normal_price_per_ebit = get_normal_price(aapl_ebit.get_per_share_data(), 
                                              calc_avg_price_ratio(aapl_quote_history, aapl_ebit.get_per_share_data()))
aapl_normal_price_per_ebit

{datetime.datetime(2000, 9, 1, 0, 0): 1.0283256886664776,
 datetime.datetime(2001, 9, 1, 0, 0): 0,
 datetime.datetime(2002, 9, 1, 0, 0): 0.0899613133594992,
 datetime.datetime(2003, 9, 1, 0, 0): 0.022880450266220514,
 datetime.datetime(2004, 9, 1, 0, 0): 0.32055368679561197,
 datetime.datetime(2005, 9, 1, 0, 0): 1.276256208241026,
 datetime.datetime(2006, 9, 1, 0, 0): 1.8592245521129165,
 datetime.datetime(2007, 9, 1, 0, 0): 3.296249882940634,
 datetime.datetime(2008, 9, 1, 0, 0): 6.139060384148234,
 datetime.datetime(2009, 9, 1, 0, 0): 8.607881987459832,
 datetime.datetime(2010, 9, 1, 0, 0): 13.224649728349727,
 datetime.datetime(2011, 9, 1, 0, 0): 24.294717555099833,
 datetime.datetime(2012, 9, 1, 0, 0): 39.23954218936427,
 datetime.datetime(2013, 9, 1, 0, 0): 35.90898668908029,
 datetime.datetime(2014, 9, 1, 0, 0): 40.967227321254306,
 datetime.datetime(2015, 9, 1, 0, 0): 58.87550052643052,
 datetime.datetime(2016, 9, 1, 0, 0): 53.18666743299383,
 datetime.datetime(2017, 9, 1, 0, 0)

## Chart price vs normal price (per metric X) 

In [10]:
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [11]:
def plot(quote_history_data, normal_price_data):
    """
    This helper function
    """
    
    # create new plot
    p = figure(title = "Price Correlated with Fundamentals",
               x_axis_type = "datetime",
               x_axis_label= "Time",
               y_axis_label= "Price")

    # add a line for historical prices by month
    p.line(list(quote_history_data.keys()), 
           list(quote_history_data.values()), 
           legend_label='Price', 
           color='black',
           line_width=2)

    # add a line for "normal prices" by year
    p.line(list(normal_price_data.keys()),
           list(normal_price_data.values()),
           legend_label = 'Normal Price',
           line_width = 2)

    show(p)
    

# testing
plot(aapl_quote_history, aapl_normal_price_per_ebit)

## Experiment

In [12]:
# parameters - choose a company
symbol = 'CAT'

# get financial history
financials_history = get_guru_data(symbol, 'financials')

In [13]:
# parameters - experiment

# set years interval for analysis
analysis_interval_years = 20

# financials parameters
section_name = 'income_statement'
metric_name = 'EBITDA'

In [14]:
# get the earlist fiscal year accordingly
start_year = datetime.utcnow().year - analysis_interval_years
start_date_quote_history = datetime(start_year, 1, 1)
start_year_financials_history = start_year - 1

# get quote history
quote_history = get_quote_history(symbol, start=start_date_quote_history.strftime('%Y-%m-%d'),
                                  end=datetime.utcnow().strftime('%Y-%m-%d'))

# get revenue and revenue per share
revenue = FinancialMetric(name='revenue',
                          timestamps=financials_history['financials']['annuals']['Fiscal Year'],
                          start_year=start_year_financials_history,
                          values=financials_history['financials']['annuals']['income_statement']['Revenue'])
revenue_per_share = FinancialMetric(name='revenue per share',
                                    timestamps=financials_history['financials']['annuals']['Fiscal Year'],
                                    start_year=start_year_financials_history,
                                    values=financials_history['financials']['annuals']['per_share_data_array']['Revenue per Share'])

# get per share metric data
metric_X = FinancialMetric(name = metric_name, 
                           timestamps = financials_history['financials']['annuals']['Fiscal Year'],
                           start_year = start_year_financials_history,
                           values = financials_history['financials']['annuals'][section_name][metric_name])

metric_X.set_per_share_data(x_total=revenue.values, x_per_share=revenue_per_share.values)

# get normal prices for metric X
normal_price_per_X = get_normal_price(metric_X.get_per_share_data(), 
                                      calc_avg_price_ratio(quote_history, metric_X.get_per_share_data()))

# plot
plot(quote_history, normal_price_per_X)

# <font color=red>Severe data issues with per share financials from Guru</font>

- It seems like the per share financials from Guru is not always properly split adjusted
- For example, there was a 2-to-1 split in 2004 for "CAT"
- And the per share "EBIDTA" value for 2004 from the Guru API (see below) are clearly not properly adjusted
- The # of outstanding shares data for 2004 is also obviously wrong
- **Next steps** - test a few more companies, summarize the pattern, and see if there's anyway to address the issue

In [15]:
financials_history['financials']['annuals']['income_statement'].keys()

dict_keys(['Revenue', 'Cost of Goods Sold', 'Gross Profit', 'Gross Margin %', 'Selling, General, & Admin. Expense', 'Research & Development', 'Other Operating Expense', 'Total Operating Expense', 'Operating Income', 'Operating Margin %', 'Interest Income', 'Interest Expense', 'Net Interest Income', 'Other Income (Expense)', 'Pretax Income', 'Tax Provision', 'Tax Rate %', 'Net Income (Continuing Operations)', 'Net Income (Discontinued Operations)', 'Other Income (Minority Interest)', 'Net Income', 'Net Margin %', 'Preferred Dividends', 'EPS (Basic)', 'EPS (Diluted)', 'Shares Outstanding (Diluted Average)', 'EBIT', 'Depreciation, Depletion and Amortization', 'EBITDA'])

In [16]:
# get oustanding shares by year
dict(zip(financials_history['financials']['annuals']['Fiscal Year'], financials_history['financials']['annuals']['income_statement']['Shares Outstanding (Diluted Average)']))

{'1991-12': '808',
 '1992-12': '807.629',
 '1993-12': '857.895',
 '1994-12': '822.4',
 '1995-12': '800',
 '1996-12': '777.778',
 '1997-12': '762.014',
 '1998-12': '736.253',
 '1999-12': '718.8',
 '2000-12': '697.351',
 '2001-12': '694.2',
 '2002-12': '693.8',
 '2003-12': '702.7',
 '2004-12': '1414.8',
 '2005-12': '705.8',
 '2006-12': '683.8',
 '2007-12': '659.5',
 '2008-12': '627.9',
 '2009-12': '626',
 '2010-12': '650.4',
 '2011-12': '666.1',
 '2012-12': '669.6',
 '2013-12': '658.6',
 '2014-12': '628.9',
 '2015-12': '601.3',
 '2016-12': '584.3',
 '2017-12': '599.3',
 '2018-12': '599.4',
 '2019-12': '567.5',
 '2020-12': '548.6',
 'TTM': '552.1'}

In [17]:
# get EBIDTA per share by year
dict(zip(financials_history['financials']['annuals']['Fiscal Year'], financials_history['financials']['annuals']['per_share_data_array']['EBITDA per Share']))

{'1991-12': '0.522',
 '1992-12': '0.848',
 '1993-12': '1.709',
 '1994-12': '2.83',
 '1995-12': '3.32',
 '1996-12': '3.835',
 '1997-12': '4.631',
 '1998-12': '4.899',
 '1999-12': '4.172',
 '2000-12': '4.943',
 '2001-12': '3.572',
 '2002-12': '4.517',
 '2003-12': '5.043',
 '2004-12': '2.885',
 '2005-12': '9.076',
 '2006-12': '11.348',
 '2007-12': '12.444',
 '2008-12': '12.594',
 '2009-12': '6.931',
 '2010-12': '9.823',
 '2011-12': '14.484',
 '2012-12': '17.198',
 '2013-12': '13.179',
 '2014-12': '10.811',
 '2015-12': '11.628',
 '2016-12': '6.295',
 '2017-12': '12.498',
 '2018-12': '18.338',
 '2019-12': '19.048',
 '2020-12': '12.652',
 'TTM': '15.588'}

# <font color=red>Severe data issues with quote data and next steps</font> 

- The historical quotes returned by the yfinance API (by RAN AROUSSI) are not correct for dates in earlier years
    - For example, see the tests below for 'CAT'. I also observed issues for at least 'EPD'.
- **Next steps**: test and validate other options to get historical quotes, such as the 'yahoo_fin' library
    - See a guide here for the 'yahoo_fin' library - https://algotrading101.com/learn/yahoo-finance-api-guide/

In [18]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [19]:
_stock = yf.Ticker('CAT')
_stock.history(interval='1mo', 
               start=start_date_quote_history.strftime('%Y-%m-%d'),
               end=datetime.utcnow().strftime('%Y-%m-%d'))['Close']

Date
2001-01-01     12.829394
2001-01-18           NaN
2001-02-01     12.161141
2001-03-01     12.973835
2001-04-01     14.675226
2001-05-01     15.944772
2001-06-01     14.734780
2001-07-01     16.221508
2001-07-18           NaN
2001-08-01     14.816905
2001-09-01     13.275945
2001-10-01     13.252237
2001-11-01     14.157962
2001-12-01     15.600038
2002-01-01     15.011864
2002-02-01     16.695044
2002-03-01     17.098053
2002-04-01     16.427370
2002-05-01     15.819819
2002-06-01     14.815000
2002-07-01     13.528711
2002-08-01     13.315654
2002-09-01     11.356750
2002-10-01     12.464355
2002-11-01     15.364480
2002-12-01     14.077428
2003-01-01     13.541674
2003-02-01     14.579709
2003-03-01     15.262156
2003-04-01     16.316866
2003-05-01     16.285427
2003-06-01     17.381529
2003-07-01     21.069572
2003-08-01     22.565790
2003-09-01     21.626474
2003-10-01     23.021318
2003-10-16           NaN
2003-11-01     24.004356
2003-12-01     26.204365
2004-01-01     24.66

In [20]:
_stock.splits

Date
2005-07-14    2.0
Name: Stock Splits, dtype: float64

In [21]:
_stock.history(interval='1mo', 
               start=start_date_quote_history.strftime('%Y-%m-%d'),
               end=datetime.utcnow().strftime('%Y-%m-%d'))

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2001-01-01,13.708479,14.397530,11.550663,12.829396,100271800.0,0.000,0.0
2001-01-18,NaN,NaN,NaN,NaN,NaN,0.170,0.0
2001-02-01,12.927060,13.155081,11.620322,12.161141,54466000.0,0.000,0.0
2001-03-01,12.161143,14.032089,11.737258,12.973836,99983000.0,0.000,0.0
2001-04-01,12.973841,15.052345,12.131915,14.675232,90777200.0,0.000,0.0
2001-05-01,14.778947,16.730828,14.408000,15.944776,105511600.0,0.000,0.0
2001-06-01,15.944774,16.633673,14.572864,14.734784,92980200.0,0.000,0.0
2001-07-01,14.734784,16.339271,14.357950,16.221510,96153000.0,0.000,0.0
2001-07-18,NaN,NaN,NaN,NaN,NaN,0.175,0.0
